# Synthetically Generated Data

We test the potential of our estimation algorithm by synthetically generated data.

In [ ]:
# std library
import os
import json
from pathlib import Path
from pprint import pprint
from datetime import datetime, timedelta
from typing import Sequence

# third-party
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from numpy import typing as npt

# local
from synthetic_data import synthetic_data_simulation  # type: ignore

### 1. Model Setting

In [ ]:
# Capacities & Total Innovation Uncertainty
param_theta = 1.0

param_c_i = 1.2 / 4
param_c_j = 1.5 / 4
param_sigma = 2.0
param_lambda = 1.0
param_r = 15.0

# Contest Time Duaration
start_time = datetime(2025, 1, 1, 0, 0, 0)
end_time = datetime(2026, 1, 1, 0, 0, 0)
contest_days = (end_time - start_time).days
contest_seconds = int((end_time - start_time).total_seconds())
contest_hours = int(contest_seconds / 3600)
contest_timegrids: list[datetime] = np.arange(start_time, end_time, \
		timedelta(hours=1), dtype=datetime).tolist()
time_unit_2f = 1 / 24

print(contest_days, contest_hours, len(contest_timegrids))

### 2. Simulations

In [ ]:
random_seed = 1000

time_grids, i_effort_dynamic, j_effort_dynamic, \
real_gap_dynamic, perceived_gap_dynamic, observed_gap_dynamic, \
observed_i_commits, observed_j_commits = synthetic_data_simulation(
	theta=param_theta,
	c_i=param_c_i,
	c_j=param_c_j,
	sigma=param_sigma,
	lamb=param_lambda,
	intensity_effort_ratio=param_r,
	hour_arrival_ub=1.0,
	start_time=start_time,
	end_time=end_time,
	time_unit=timedelta(hours=1),
	time_unit_2f=time_unit_2f,
	approx = True,  # using approximated version by default
	seed_brownian = random_seed + 0,
	seed_poisson = random_seed + 1,
	seed_uniform = random_seed + 2,
)

In [ ]:
# pprint(observed_gap_dynamic)
pprint(f'number of i submissions = {len(observed_i_commits)}')
pprint(f'number of j submissions = {len(observed_j_commits)}')

In [ ]:
def plot_ts_events(
		time_grids: Sequence[datetime] | npt.NDArray[np.datetime64],
		ts: dict[str, Sequence | npt.NDArray],
		events: dict[str, Sequence | npt.NDArray],
		*,
		figsize=(10, 5),
		height_ratios=[3, 1],
):
	fig, (ax_main, ax_events) = plt.subplots(
		nrows=2, ncols=1, figsize=figsize, height_ratios=height_ratios
	)
	# plot time series
	for ts_key, ts_val in ts.items():
		ax_main.plot(time_grids, ts_val, label=ts_key)
	ax_main.legend()
	xrange = ax_main.get_xlim()
	# plot events
	event_height = 0.2
	ytick = 0
	yticks = []
	yticklabels = []
	for event_key, event_val in events.items():
		yticks.append(ytick)
		yticklabels.append(event_key)
		ax_events.hlines(y=ytick, xmin=xrange[0], xmax=xrange[1])
		ax_events.vlines(x=event_val, ymin=ytick - event_height, ymax=ytick + event_height)
		ytick -= 2.5 * event_height
	ax_events.set_xlim(xrange)
	ax_events.get_xaxis().set_visible(False)
	ax_events.set_yticks(yticks)
	ax_events.set_yticklabels(yticklabels)
	return fig, ax_main, ax_events

In [ ]:
fig, _, _ = plot_ts_events(time_grids, {
			'real': real_gap_dynamic[:-1],
			'observed': observed_gap_dynamic[:-1],
			'perceived': perceived_gap_dynamic[:-1],
			#'effort gap (cumsum)': (i_effort_dynamic - j_effort_dynamic).cumsum(),
		}, {
			'player i': observed_i_commits,
			'player j': observed_j_commits,
	})

In [ ]:
plt.figure(figsize=(10, 3))
plt.plot(np.array(time_grids), i_effort_dynamic, label='player i')
plt.plot(np.array(time_grids), j_effort_dynamic, label='player j')
plt.legend()
plt.show()

In [ ]:
# save the observed data
wd = os.getcwd()
wd_synthetic_data = os.path.join(wd, f'data_{contest_days}.json')
roundint = lambda x: int(round(x))

with open(wd_synthetic_data, 'w') as f:
	json.dump({
		'theta': param_theta,
		'ratio': param_r,
		'N_Delta': roundint((end_time - start_time).total_seconds() / 3600),
		'Delta2f': time_unit_2f,
		'Ni': len(observed_i_commits),
		'Nj': len(observed_j_commits),
		'hat_t_i': [(dt - start_time).total_seconds() / 3600 for dt in observed_i_commits],
		'hat_t_j': [(dt - start_time).total_seconds() / 3600 for dt in observed_j_commits],
		'hat_y': observed_gap_dynamic.tolist(),
		'efforts_i': i_effort_dynamic.tolist(),
		'efforts_j': j_effort_dynamic.tolist(),
	}, f, indent=4)

### 3. Bayesian Inference

In [ ]:
# import cmdstanpy
# cmdstanpy.install_cmdstan()
from cmdstanpy import CmdStanModel, compile_stan_file

In [ ]:
# build stan model
stan_file = os.path.join(wd, f'synthetic_data.stan')
output_dir = Path('./tmp')
exe_path = compile_stan_file(
	src=stan_file,
	cpp_options={
		"LDFLAGS": "-headerpad_max_install_names",
		"CXXFLAGS": "-O3",
})
model = CmdStanModel(stan_file=stan_file, exe_file=exe_path)

In [ ]:
# fit the model with data
fit = model.sample( \
		data=wd_synthetic_data,
		iter_warmup=1000,
		iter_sampling=2000,
		chains=4,
		parallel_chains=4,
		show_console=False,
		#max_treedepth=12,  # for difficult model
		#adapt_delta=0.99,  # for difficult model
		output_dir=output_dir,
		seed=12345,
	)

In [ ]:
posteriors = fit.stan_variables()

In [ ]:
def rmse(true, mean, std):
	return ((true - mean)**2 + std**2)**0.5

def display_estimation_results(var_names: list[str], true_vals, posteriors):
	data = []
	for name, true_val in zip(var_names, true_vals):
		posterior_mean = posteriors[name].mean()
		posterior_std = posteriors[name].std()
		quantile_05, quantile_95 = np.percentile(posteriors[name], [2.5, 97.5])
		record = (
			name, true_val, posterior_mean, posterior_std,
			rmse(true_val, posterior_mean, posterior_std),
			quantile_05, quantile_95
		)
		data.append(record)
	columns = ['Name', 'True Val.', 'Posterior Mean', 'Posterior Std.', 'RMSE', '2.5% Quantile', '97.5% Quantile']
	return pd.DataFrame(data, columns=columns)

In [ ]:
display_estimation_results(
	['c_i', 'c_j', 'r', 'sigma', 'lambda', 'mu_0'],
	[param_c_i, param_c_j, param_r, param_sigma, param_lambda, 0],
	posteriors
)

Compare posterior `m[1]..m[T]` with real

In [ ]:
posterior_m_i_mean = fit.stan_variable('m_i').mean(axis=0)
posterior_m_j_mean = fit.stan_variable('m_j').mean(axis=0)

plt.figure(figsize=(10, 3))
plt.plot(np.array(time_grids), posterior_m_i_mean, label='posterior m_i')
plt.plot(np.array(time_grids), i_effort_dynamic, label='real m_i', linestyle='dotted')
plt.legend()
plt.show()
print(np.sum(posterior_m_i_mean - i_effort_dynamic))

plt.figure(figsize=(10, 3))
plt.plot(np.array(time_grids), posterior_m_j_mean, label='posterior m_j')
plt.plot(np.array(time_grids), j_effort_dynamic, label='real m_j', linestyle='dotted')
plt.legend()
plt.show()
print(np.sum(posterior_m_j_mean - j_effort_dynamic))

In [ ]:
## compare posterior \tilde{y}[1]..\tilde{y}[T] with real

posterior_tilde_y_mean = fit.stan_variable('tilde_y').mean(axis=0)

plt.figure(figsize=(10, 3))
plt.plot(np.array(time_grids), posterior_tilde_y_mean[:-1], label=r'$\tilde{y}_t$ (posterior)')
plt.plot(np.array(time_grids), perceived_gap_dynamic[:-1], label=r'$\tilde{y}_t$ (real)', linestyle='dotted')
plt.legend()
plt.show()
print(np.sum(posterior_tilde_y_mean - perceived_gap_dynamic))

### Clean Up

In [ ]:
for file in output_dir.iterdir():
	if file.is_file():
		file.unlink()